# Tutorial for Bokeh client visualization
* import libraries
* generate random data source ABCD uniform
* simple case with no widgets
* adding selection widgets
* adding widgets for some visualization parameters

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import bokehDrawSA
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata
import pandas as pd
import numpy as np
import math
import logging
output_notebook()


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

- Create a random data source

In [ ]:
#init test random data
df = pd.DataFrame(np.random.random_sample(size=(2000, 6)), columns=list('ABCDEF'))
initMetadata(df)
mapDDC={0:"A0",1:"A1",2:"A2",3:"A3",4:"A4"}
df["B"]=np.linspace(0,1,2000)
df.eval("Bool=A>0.5", inplace=True)
df.eval("BoolB=B>0.5", inplace=True)
df.eval("BoolC=C>0.1", inplace=True)
df["A"]=df["A"].round(3);
df["B"]=df["B"].round(3);
df["C"]=df["C"].round(3);
df["D"]=df["D"].round(3);
df["AA"]=((df.A*10).round(0)).astype(pd.CategoricalDtype(ordered=True))
df["CC"]=((df.C*5).round(0)).astype(int)
df["DD"]=((df.D*4).round(0)).astype(int)
df["DDC"]=((df.D*4).round(0)).astype(int).map(mapDDC)
df["EE"]=(df.E*4).round(0)
df['errY']=df.A*0.02+0.02;
df.loc[15, "A"] = math.nan
df.head(10)
df.meta.metaData = {'A.AxisTitle': "A (cm)", 'B.AxisTitle': "B (cm/s)", 'C.AxisTitle': "C (s)", 'D.AxisTitle': "D (a.u.)", 'Bool.AxisTitle': "A>half", 'E.AxisTitle': "Category"}


* Identify the layout
* Draw the graphs using bokehDrawArray - 3D doesn't work in jupyter notebook
* Layout description can be a dictionary, this way it creates multiple tabs

In [ ]:
figureArray = [
#   ['A'], ['C-A'], {"color": "red", "size": 7, "colorZvar":"C", "filter": "A<0.5"}],
    [['A'], ['A*A-C*C'], {"color": "red", "size": 2, "colorZvar": "A", "varZ": "C"}],
    [['A'], ['C+A', 'C-A', 'A/A']],
    [['B'], ['C+B', 'C-B'], { "colorZvar": "D", "errY": "errY", "rescaleColorMapper": True}],
    [['D'], ['(A+B+C)*D'], {"colorZvar": "D", "errY": "errY"} ],
    [['D'], ['D*10'], {"errY": "errY"}],
    {"size":2, "legend_options": {"label_text_font_size": "13px"}}
]
layout = {
    "A": [
        [0, 1, 2, {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 300}],
        {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2}
        ],
    "B": [
        [3, 4, {'commonX': 1, 'y_visible': 3, 'x_visible':1, 'plot_height': 100}],
        {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2}
        ]
}
tooltips = [("VarA", "(@A)"), ("VarB", "(@B)"), ("VarC", "(@C)"), ("VarD", "(@D)")]
pAll = bokehDrawArray(df, "A>0", figureArray, layout=layout, size=4, tooltips=tooltips)
show(pAll[0])

In [ ]:
logging.disable(1)

* Add sliders for selection
* Possible sliders: 
    * range slider
    * slider
    * multiSelect
    * select
* widget layout description works same as figure layout description

In [ ]:
output_file("test2.html")

widgetParams=[
    ['range', ['A']],
    ['range', ['B', 0, 1, 0.1, 0, 1]],

    ['range', ['C'], {'type': 'minmax'}],
    ['range', ['D'], {'type': 'sigma', 'bins': 10, 'sigma': 3}],
    ['range', ['E'], {'type': 'sigmaMed', 'bins': 10, 'sigma': 3}],
    ['slider', ['AA'], {'bins': 10}],
    ['multiSelect', ["DDC"]],
    ['select',["CC", 0, 1, 2, 3],  {"default": 1}],
    ['multiSelect',["BoolB"]],
]
widgetLayoutDesc={
    "Selection": [[0, 1, 2], [3, 4], [5, 6],[7,8], {'sizing_mode': 'scale_width'}]
    }    
bokehDrawSA.fromArray(df, "A>0", figureArray, widgetParams, layout=layout, tooltips=tooltips, widgetLayout=widgetLayoutDesc)

* Visualization parameters
* So far only controllable by sliders and selects
* If it controls a parameter, it needs "callback":"parameter" in options. This will probably be fixed soon, making the parameter redundant.
* parameterArray options:
    * name - the name it is indexed by in figureArray / aliasArray
    * value - the initial value - because of a bug, if using a select to control the client side parameter, the "default" option has to be specified, otherwise it will be initialized to the first in the options list
    * range - if controlled by a slider, the range the variable can take
    * options - the options the parameter can have as its value
* Controllable by parameterArray:
    * color axis
    * marker size
    * legend options - in this example we set the legend font size
    * functions in aliasArray

In [ ]:
parameterArray = [
    {"name": "colorZ", "value":"EE", "options":["A", "B", "DD", "EE"]},
    {"name": "size", "value":7, "range":[0, 30]},
    {"name": "legendFontSize", "value":"13px", "options":["9px", "11px", "13px", "15px"]},
]

figureArray = [
    [['A'], ['A*A-C*C'], {"color": "red", "size": 2, "colorZvar": "A", "varZ": "C", "errY": "errY", "errX":"0.01"}],
    [['A'], ['C+A', 'C-A', 'A/A']],
    [['B'], ['C+B', 'C-B'], { "colorZvar": "colorZ", "errY": "errY", "rescaleColorMapper": True}],
    [['D'], ['(A+B+C)*D'], {"colorZvar": "colorZ", "size": 10, "errY": "errY"} ],
    [['D'], ['D*10'], {"errY": "errY"}],
    {"size":"size", "legend_options": {"label_text_font_size": "legendFontSize"}}
]
widgetParams=[
    ['range', ['A']],
    ['range', ['B', 0, 1, 0.1, 0, 1]],

    ['range', ['C'], {'type': 'minmax'}],
    ['range', ['D'], {'type': 'sigma', 'bins': 10, 'sigma': 3}],
    ['range', ['E'], {'type': 'sigmaMed', 'bins': 10, 'sigma': 3}],
    ['slider', ['AA'], {'bins': 10}],
    ['multiSelect', ["DDC"]],
    ['select',["CC", 0, 1, 2, 3]],
    ['multiSelect',["BoolB"]],
    #['slider','F', ['@min()','@max()','@med','@min()','@median()+3*#tlm()']], # to be implmneted
    ['select',["colorZ"], {"callback": "parameter", "default": 3}],
    ['slider',["size"], {"callback": "parameter"}],
    ['select',["legendFontSize"], {"callback": "parameter", "default": 2}],
]
widgetLayoutDesc={
    "Selection": [[0, 1, 2], [3, 4], [5, 6],[7,8], {'sizing_mode': 'scale_width'}],
    "Graphics": [[9, 10, 11], {'sizing_mode': 'scale_width'}]
    }
figureLayoutDesc={
    "A": [
        [0, 1, 2, {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 300}],
        {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2}
        ],
    "B": [
        [3, 4, {'commonX': 1, 'y_visible': 3, 'x_visible':1, 'plot_height': 100}],
        {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2}
        ]
}
bokehDrawSA.fromArray(df, "A>0", figureArray, widgetParams, layout=layout, tooltips=tooltips, widgetLayout=widgetLayoutDesc, parameterArray=parameterArray)